In [ ]:
import time
import requests
import random
import base64
import string
import hashlib
import os
import json
import re
import csv
from urllib.parse import  urlencode
appid="申请的app_id"
appkey="申请的app_key"

In [ ]:
#定义一个图片转base64编码的函数
def to_base64(path):
    with open(path,'rb') as f:
            base64_data = base64.b64encode(f.read())
            return base64_data

In [ ]:
def get_params(appid,appkey,img):
    time_stamp=str(int(time.time()))#将当前时间戳转换为秒级的字符串
    random_str=''.join(random.sample(string.ascii_letters+string.digits,18))#生成随机的字符串
    params={'app_id':appid,
           'time_stamp':time_stamp,
           'nonce_str':random_str,
           'mode':'0',
           'image':img}
    sort_dict= sorted(params.items(), key=lambda item:item[0], reverse = False)  #字典排序
    sort_dict.append(('app_key',appkey))#尾部添加appkey
    rawtext= urlencode(sort_dict).encode() #urlencod编码
    sha = hashlib.md5()    
    sha.update(rawtext)
    md5text= sha.hexdigest().upper()#MD5加密计算,并大写
    params['sign']=md5text  #将签名赋值到sign
    return  params          

In [ ]:
def mian(appid,appkey,path):
    data={}
    img=to_base64(path)
    params=get_params(appid,appkey,img)#获得提交的参数
    url='https://api.ai.qq.com/fcgi-bin/face/face_detectface'
    response=requests.post(url,data=params)
    js=json.loads(response.text)
    try:
        js1=js.get('data').get('face_list')
        js2=js1[0]['beauty']
        data['name']=re.findall('(.*?).jpg',path)[0]
        data['beauty']=js2
        data['age']=js1[0]['age']
        return data
    except:
        return data   

In [ ]:
path1=r"C:\Users\约明\Desktop\测试"#输入要测试的文件夹
os.chdir(path1)
with open('txAI.csv','w',newline='') as f:#最终结果储存在输入的文件夹中
        title=['name','age','beauty']
        writer=csv.DictWriter(f,fieldnames=title)
        writer.writeheader()
        for file in os.listdir(path1):
            data=mian(appid,appkey,file)
            writer.writerow(data)
            print('正确的结果：'+str(data))
    